In [1]:
import sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

import pandas as pd

# 0. load data

In [2]:
q = '''select
            *
        from transactions
        where
            status='A' and
            (date='2023-11-28' or (date='2023-11-29' and hour<3))'''

df_2023 = redshift_query_read(q, schema='production')

In [3]:
q = '''select 
            * 
        from transactions 
        where
            status='A' and
            (date='2022-11-29' or (date='2022-11-30' and hour<3))'''
df_2022 = redshift_query_read(q, schema='production')

In [4]:
q = '''select
            *
        from transactions
        where
            status='A' and
            (date='2021-11-30' or (date='2021-12-01' and hour<3))'''
df_2021 = redshift_query_read(q, schema='production')

In [5]:
df = pd.concat([df_2021, df_2022, df_2023])

In [6]:
df.columns

Index(['id', 'org', 'form', 'status', 'amount', 'donations_amt',
       'purchases_amt', 'events_amt', 'registrations_amt', 'events_tickets',
       'purchases_quantity', 'donations_count', 'events_count',
       'purchases_count', 'registrations_count', 'fields', 'event_fieds',
       'recurring', 'payment_type', 'creatingtransactionfor',
       'transdonationentity', 'transdonationentitytype', 'source', 'source_id',
       'date', 'hour', 'day', 'month', 'year', 'zip', 'state', 'email',
       'matchinggifts_count', 'smspledge_count', 'smspledge_amt',
       'auctionpurchase_count', 'auctiondonation_count', 'is_fraud', 'channel',
       'useragent', 'gift_assist_count', 'gift_assist_amt', 'qgiv_fee',
       'platform', 'recurring_origin', 'is_new_form', 'isexpresscheckout'],
      dtype='object')

In [7]:
for year in df['year'].unique():
    _df = df[df['year']==year]
    
    print("{}".format(year))
    print("transactions: {:,}".format(len(_df)))
    print("orgs: {:,}".format(len(_df['org'].unique())))
    print("forms: {:,}".format(len(_df['form'].unique())))
    print("mean onetime: ${:.2f}".format(_df[_df['recurring']==0]['amount'].mean()))
    print("median onetime: ${:.2f}".format(_df[_df['recurring']==0]['amount'].median()))
    ec_perc = len(_df[_df['isexpresscheckout']]) / len(_df)
    print("Express checkout: {:.2f}%".format(ec_perc * 100.))
    ga_perc = len(_df[_df['gift_assist_count']>0]) / len(_df)
    print("Gift assist: {:.2f}%".format(ga_perc * 100.))
    print("Purchases: ${:,.2f}".format(_df['purchases_amt'].sum()))
    print("Events: ${:,.2f}".format(_df['events_amt'].sum()))
    print("P2P registrations: ${:,.2f}".format(_df['registrations_amt'].sum()))
    
    mobile_perc = len(_df[(_df['platform']=='iPhone')|(_df['platform']=='Android')|(_df['platform']=='iPad')]) / len(_df)
    mobile_vol = _df[(_df['platform']=='iPhone')|(_df['platform']=='Android')|(_df['platform']=='iPad')]['amount'].sum()
    mobile_mean = _df[(_df['platform']=='iPhone')|(_df['platform']=='Android')|(_df['platform']=='iPad')]['amount'].mean()
    mobile_median = _df[(_df['platform']=='iPhone')|(_df['platform']=='Android')|(_df['platform']=='iPad')]['amount'].median()
    print("Mobile perc: {:.2f}%".format(mobile_perc * 100.))
    print("Mobile amount: ${:,.2f}".format(mobile_vol))
    print("Mobile mean: ${:,.2f}".format(mobile_mean))
    print("Mobile median: ${:,.2f}".format(mobile_median))
    
    print("platforms:")
    print(_df['platform'].value_counts(normalize=True))
    print()
    
    print("source:")
    print(_df['source'].value_counts(normalize=True))
    print()
    
    print("payment type:")
    print(_df['payment_type'].value_counts(normalize=True))
    print()
    

2021
transactions: 41,458
orgs: 1,827
forms: 3,631
mean onetime: $186.38
median onetime: $60.00
Express checkout: 0.00%
Gift assist: 41.27%
Purchases: $8,031.00
Events: $63,276.71
P2P registrations: $3,090.00
Mobile perc: 26.54%
Mobile amount: $1,686,501.63
Mobile mean: $153.28
Mobile median: $52.00
platforms:
Windows    0.328694
           0.221887
iPhone     0.186791
Mac        0.184018
Android    0.065995
iPad       0.012615
Name: platform, dtype: float64

source:
don_form    0.637320
mobile      0.178422
p2p         0.133557
vt          0.039558
sms         0.005234
fb          0.003908
givi        0.000941
kiosk       0.000772
mobilevt    0.000289
Name: source, dtype: float64

payment type:
VS    0.578185
MC    0.212429
AM    0.091427
PP    0.043669
DC    0.043133
ck    0.027141
FB    0.003895
BC    0.000122
Name: payment_type, dtype: float64

2022
transactions: 42,165
orgs: 1,915
forms: 3,937
mean onetime: $182.32
median onetime: $52.50
Express checkout: 0.00%
Gift assist: 43.82%